In [1]:
import os

In [2]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\Kidney-Disease-Classification'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataINgestionConfig:
    root_dir: Path
    source_URL: str
    local_source_file: Path
    unzip_dir: Path


In [14]:
from kidneyDiseaseClassifier.constants import *
from kidneyDiseaseClassifier.utils.common import create_directories, read_yaml


In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataINgestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataINgestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_source_file = config.local_source_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
    
    


In [16]:
import os
import zipfile
import gdown
from kidneyDiseaseClassifier import logger
from kidneyDiseaseClassifier.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataINgestionConfig):
        self.config = config

    def download_file(self):

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_source_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading dataset from {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Dataset downloaded successfully to {zip_download_dir}")

        except Exception as e:
            logger.error(f"Error occurred while downloading dataset: {str(e)}")


    def unzip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_source_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logger.info(f"Dataset unzipped successfully to {unzip_path}")
        
        except Exception as e:
            raise e

    


In [22]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2025-06-30 12:45:45,670: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-30 12:45:45,675: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-30 12:45:45,676: INFO: common: Created file directory at: artifacts]
[2025-06-30 12:45:45,676: INFO: common: Created file directory at: artifacts/data_ingestion]
[2025-06-30 12:45:45,679: INFO: 4166140176: Downloading dataset from https://drive.google.com/file/d/1prK7BFC7UgLg1yKNiLB4-CvOlPZzycbN/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1prK7BFC7UgLg1yKNiLB4-CvOlPZzycbN
From (redirected): https://drive.google.com/uc?export=download&id=1prK7BFC7UgLg1yKNiLB4-CvOlPZzycbN&confirm=t&uuid=bd4658e3-c5c8-4dce-9c59-d842d4f9ff32
To: c:\Users\abhay\Desktop\ML Project\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:08<00:00, 6.71MB/s]


[2025-06-30 12:45:56,799: INFO: 4166140176: Dataset downloaded successfully to artifacts/data_ingestion/data.zip]
[2025-06-30 12:45:57,482: INFO: 4166140176: Dataset unzipped successfully to artifacts/data_ingestion]
